# Initialisation
Select the mode in `pandas`, `cudf`, `dask` or `dask-cudf`.

In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["MODE"]="pandas"  # Reset the kernel if you change this

#import cardif_dask as vdf  # Import Virtual Dataframe
import virtual_dataframe as vdf
import pandas as pd
import cupy as cp

# Object Creation
Creating a `VSeries`

In [2]:
s = vdf.VSeries([1,2,3,None,4],npartitions=2)
s.compute()

0    1.0
1    2.0
2    3.0
3    NaN
4    4.0
dtype: float64

Creating a `VDataFrame` by specifying values for each column.

In [3]:
df = vdf.VDataFrame({'a': list(range(20)),
                 'b': list(reversed(range(20))),
                 'c': list(range(20))
                }, npartitions=2)
df.compute()

,a,b,c
0,0,19,0
1,1,18,1
2,2,17,2
3,3,16,3
4,4,15,4
5,5,14,5
6,6,13,6
7,7,12,7
8,8,11,8
9,9,10,9


Creating a `VDataFrame` from a pandas `Dataframe`.

> Note that best practice for VDataFrame is to read data directly into a ̀`VDataFrame` with something like `read_csv()` (discussed below).

In [4]:
pdf = pd.DataFrame({'a': [0, 1, 2, 3],'b': [0.1, 0.2, None, 0.3]})
df = vdf.from_pandas(pdf,npartitions=2)
df.compute()

,a,b
0,0,0.1
1,1,0.2
2,2,NaN
3,3,0.3


# Viewing Data
Viewing the top rows of a `VDataFrame`.

In [5]:
df.head(2).compute()

,a,b
0,0,0.1
1,1,0.2


In [6]:
df.sort_values(by='b').compute()

,a,b
0,0,0.1
1,1,0.2
3,3,0.3
2,2,NaN


# Selection
## Getting
Selecting a single column, which initially yields a `VSeries`.

In [7]:
df['a'].compute()

0    0
1    1
2    2
3    3
Name: a, dtype: int64

## Selection by Label
Selecting rows from index 2 to index 5 from columns ‘a’ and ‘b’.

In [8]:
df.loc[2:5, ['a', 'b']].compute()

,a,b
2,2,NaN
3,3,0.3


## Selection by Position
Selecting via integers and integer slices, like numpy/pandas. 
> Note that this functionality is not available for `dask-cudf`.

In [9]:
if vdf.VDF_MODE != vdf.Mode.dask_cudf:
    df.iloc[0].compute()

In [10]:
df.iloc[0:3, 0:2].compute()

,a,b
0,0,0.1
1,1,0.2
2,2,NaN


You can also select elements of a `VDataFrame` or `VSeries` with direct index access.

In [11]:
df[3:5].compute()

,a,b
3,3,0.3


In [12]:
s[3:5].compute()

3    NaN
4    4.0
dtype: float64

## Boolean Indexing
Selecting rows in a `VDataFrame` or `VSeries` by direct `Boolean` indexing.

In [13]:
df[df.b > 15].compute()

,a,b


Selecting values from a `DataFrame` where a `Boolean` condition is met, via the query API.

In [14]:
df.query("b == 3").compute()

,a,b


You can also pass local variables to Dask-cuDF queries, via the local_dict keyword. 
With standard cuDF, you may either use the local_dict keyword or directly pass the variable 
via the @ keyword. Supported logical operators include >, <, >=, <=, ==, and !=.

In [15]:
value = 3
df.query("b == @value").compute()

,a,b


In [16]:
value = 3
df.query("b == @val", local_dict={'val':value}).compute()

,a,b


Using the isin method for filtering.

In [17]:
df[df.a.isin([0, 5])].compute()

,a,b
0,0,0.1


# MultiIndex
Virtual Dataframe supports hierarchical indexing of DataFrames using MultiIndex. 
Grouping hierarchically (see Grouping below) automatically produces a DataFrame with a MultiIndex.

In [18]:
# FIXME
arrays = [['a', 'a', 'b', 'b'], [1, 2, 3, 4]]
tuples = list(zip(*arrays))
idx = vdf.MultiIndex.from_tuples(tuples)
idx

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 3),
            ('b', 4)],
           )

In [19]:
import random
import array
rand4_1=array.array('d',[random.random() for i in range(0,4)])
rand4_2=array.array('d',[random.random() for i in range(0,4)])
# FIXME rand4= cp.random.rand(4)

In [20]:
df1 = vdf.VDataFrame({'first': rand4_1, 'second': rand4_2})
df1.index = idx
df1

first    second
a 1  0.310476  0.889532
  2  0.248096  0.677649
b 3  0.547354  0.181829
  4  0.516652  0.813318

In [21]:
df2 = vdf.VDataFrame({'first': rand4_1, 'second': rand4_2}).T
df2.columns = idx
df2

a                   b          
               1         2         3         4
first   0.310476  0.248096  0.547354  0.516652
second  0.889532  0.677649  0.181829  0.813318

In [22]:
df1.loc[('b', 3)]

first     0.547354
second    0.181829
Name: (b, 3), dtype: float64

# Missing Data
Missing data can be replaced by using the fillna method.

In [23]:
s.fillna(999).compute()

0      1.0
1      2.0
2      3.0
3    999.0
4      4.0
dtype: float64

# Operations
## Stats
Calculating descriptive statistics for a Series.

In [24]:
vdf.compute(s.mean())[0], vdf.compute(s.var())[0]

(2.5, 1.6666666666666667)

# Applymap
Applying functions to a Series. 
Note that applying user defined functions directly with Dask-cuDF is not yet implemented. 
For now, you can use map_partitions to apply a function to each partition of the distributed dataframe.

In [25]:
def add_ten(num):
    return num + 10

df['a'].apply(add_ten).compute()

0    10
1    11
2    12
3    13
Name: a, dtype: int64

In [26]:
# Not with pandas
if vdf.VDF_MODE in (vdf.Mode.dask, vdf.Mode.dask_cudf):
    df['a'].map_partitions(add_ten).compute()

# Histogramming
Counting the number of occurrences of each unique value of variable.

In [27]:
df.a.value_counts().compute()

0    1
1    1
2    1
3    1
Name: a, dtype: int64

# String Methods
Virtual Dataframe provides string processing methods in the str attribute of Series. 

In [28]:
s = vdf.VSeries(['A', 'B', 'C', 'Aaba', 'Baca', None, 'CABA', 'dog', 'cat'], npartitions=2)
s.str.lower().compute()

0       a
1       b
2       c
3    aaba
4    baca
5    None
6    caba
7     dog
8     cat
dtype: object

# Concat
Concatenating VSeries and VDataFrames row-wise.

In [29]:
s = vdf.VSeries([1, 2, 3, None, 5],npartitions=2)
vdf.concat([s, s]).compute()

0    1.0
1    2.0
2    3.0
3    NaN
4    5.0
0    1.0
1    2.0
2    3.0
3    NaN
4    5.0
dtype: float64

# Join
Performing SQL style merges. 
Note that the dataframe order is not maintained, but may be restored post-merge by sorting by the index.

In [30]:
df_a = vdf.VDataFrame()
df_a['key'] = ['a', 'b', 'c', 'd', 'e']
df_a['vals_a'] = [float(i + 10) for i in range(5)]

df_b = vdf.VDataFrame()
df_b['key'] = ['a', 'c', 'e']
df_b['vals_b'] = [float(i+100) for i in range(3)]

merged = df_a.merge(df_b, on=['key'], how='left').compute()
merged

,key,vals_a,vals_b
0,a,10.0,100.0
1,b,11.0,NaN
2,c,12.0,101.0
3,d,13.0,NaN
4,e,14.0,102.0


# FIXME Grouping
Virtual Dataframe support the Split-Apply-Combine groupby paradigm.

In [31]:
# FIXME
df['agg_col1'] = [1 if x % 2 == 0 else 0 for x in range(len(df))]
df['agg_col2'] = [1 if x % 3 == 0 else 0 for x in range(len(df))]

ddf = vdf.from_cudf(df, npartitions=2)


AttributeError: module 'virtual_dataframe' has no attribute 'from_cudf'

Grouping and then applying the sum function to the grouped data.

In [32]:
df.groupby('agg_col1').sum().compute()

,a,b,agg_col2
agg_col1,,,
0,4,0.5,1
1,2,0.1,1


Grouping hierarchically then applying the sum function to grouped data.

In [33]:
df.groupby(['agg_col1', 'agg_col2']).sum().compute()

a    b
agg_col1 agg_col2        
0        0         1  0.2
         1         3  0.3
1        0         2  0.0
         1         0  0.1

Grouping and applying statistical functions to specific columns, using agg.

In [34]:
# FIXME
df.groupby('agg_col1').agg({'a':'max', 'b':'mean', 'c':'sum'}).compute()

KeyError: "Column(s) ['c'] do not exist"

# Transpose
Transposing a dataframe, using either the transpose method or `T` property. 
Currently, all columns must have the same type. 

> Transposing is not currently implemented in `dask-cudf`.

In [35]:
df = vdf.VDataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
df

,a,b
0,1,4
1,2,5
2,3,6


In [36]:
df.transpose().compute()

,0,1,2
a,1,2,3
b,4,5,6


# FIXME Time Series
`VDataFrames` supports datetime typed columns, which allow users to interact with and filter data based on specific timestamps.

In [37]:
# FIXME
rand_len = cp.random.sample(len(date_df))

NameError: name 'date_df' is not defined

In [38]:
#FIXME
import datetime as dt

date_df = vdf.VDataFrame()
date_df['date'] = pd.date_range('11/20/2018', periods=72, freq='D')
date_df['value'] = rand_len

search_date = dt.datetime.strptime('2018-11-23', '%Y-%m-%d')
date_df.query('date <= @search_date')

NameError: name 'rand_len' is not defined

In [39]:
# FIXME
date_ddf = dask_cudf.from_cudf(date_df, npartitions=2)
date_ddf.query('date <= @search_date', local_dict={'search_date':search_date}).compute()

NameError: name 'dask_cudf' is not defined

# FIXME Categoricals
VDataFrames support categorical columns.

In [40]:
# FIXME
cdf = vdf.VDataFrame({"id": [1, 2, 3, 4, 5, 6], "grade":['a', 'b', 'b', 'a', 'a', 'e']}, npartitions=2)
cdf['grade'] = df['grade'].astype('category')
cdf.compute()

KeyError: 'grade'

Accessing the categories of a column. 

> Note that this is currently not supported in `dask-cudf`.

In [41]:
# FIXME
cdf.grade.cat.categories

AttributeError: Can only use .cat accessor with a 'category' dtype

Accessing the underlying code values of each categorical observation.

In [ ]:
cdf.grade.cat.codes.compute()

# Converting Data Representation
## Pandas
Converting a `VDataFrame` to a pandas DataFrame.

In [42]:
df.head().to_pandas()

,a,b
0,1,4
1,2,5
2,3,6


## Numpy
Converting a `VDataFrame` to a numpy ndarray.

In [43]:
df.to_numpy()

array([[1, 4],
       [2, 5],
       [3, 6]])

Converting a `VSeries` to a numpy ndarray.

In [44]:
df['a'].to_numpy()

array([1, 2, 3])

## Arrow
Converting a `VDataFrame` to a PyArrow Table.

> Note that this functionality is not available for `pandas` or `dask`.

In [45]:
if vdf.VDF_MODE in (vdf.Mode.cudf, vdf.Mode.dask_cudf):
    df.to_arrow()

# Getting Data In/Out
## CSV
Writing to a CSV file.

In [46]:
if not os.path.exists('example_output'):
    os.mkdir('example_output')
    
df.to_csv('example_output/foo.csv', index=False)

Reading from a csv file.

In [47]:
df = vdf.read_csv('example_output/foo.csv')
df.compute()

,a,b
0,1,4
1,2,5
2,3,6


Reading all CSV files in a directory into a single dask_cudf.DataFrame, using the star wildcard.
> Not implemented in pandas or cudf ?

In [48]:
# FIXME: extends pandas
if vdf.VDF_MODE in (vdf.Mode.dask, vdf.Mode.dask_cudf):
    df = vdf.read_csv('example_output/*.csv')
    df.compute()

# Parquet
Writing to parquet files, using the CPU via PyArrow.

In [49]:
df.to_parquet('example_output/temp_parquet')

## ORC
Reading ORC files.
> FIXME

In [50]:
if vdf.VDF_MODE in (vdf.Mode.dask, vdf.Mode.dask_cudf):
    from pathlib import Path
    cudf_root = Path(".").absolute().parents[3]
    orc_file = Path("python/cudf/cudf/tests/data/orc/TestOrcFile.test1.orc")
    file_path = cudf_root / orc_file

In [51]:
# FIXME
df2 = vdf.read_orc(file_path)
df2

AttributeError: module 'virtual_dataframe' has no attribute 'read_orc'